In [26]:
pip install pandas faker


In [27]:
import pandas as pd
from faker import Faker
import random

# Initialize Faker
fake = Faker()

# Dataset sizes and random symptom conditions
NUM_PATIENTS = 100
NUM_DOCTORS = 20
NUM_SYMPTOMS = 10
NUM_APPOINTMENTS = 200
NUM_MEDICATIONS = 50

symptoms_conditions = [
    {"symptoms": "fever and cough", "condition": "Flu"},
    {"symptoms": "chest pain and breathlessness", "condition": "Heart Attack"},
    {"symptoms": "headache and nausea", "condition": "Migraine"},
    {"symptoms": "abdominal pain and diarrhea", "condition": "Food Poisoning"},
    {"symptoms": "back pain", "condition": "Muscle Strain"},
    {"symptoms": "sore throat and fever", "condition": "Tonsillitis"},
    {"symptoms": "joint pain", "condition": "Arthritis"},
    {"symptoms": "rash and itching", "condition": "Allergy"},
    {"symptoms": "frequent urination and thirst", "condition": "Diabetes"},
    {"symptoms": "dizziness and fatigue", "condition": "Anemia"},
]

# Generate doctors, patients, medications, and appointments
doctors = [{"doctor_id": fake.uuid4(), "name": fake.name(), "specialization": random.choice(["Cardiologist", "General Physician", "Dermatologist", "Neurologist"]), "availability": f"{random.choice(['Mon', 'Tue', 'Wed', 'Thu', 'Fri'])} {random.randint(9, 12)} AM to {random.randint(1, 5)} PM"} for _ in range(NUM_DOCTORS)]

patients = [{"patient_id": fake.uuid4(), "name": fake.name(), "age": random.randint(1, 90), "gender": random.choice(["Male", "Female"]), "medical_history": random.sample([s["condition"] for s in symptoms_conditions], k=random.randint(0, 3)), "allergies": random.choice(["Penicillin", "Pollen", "None", "Peanuts"])} for _ in range(NUM_PATIENTS)]

medications = [{"medication_id": fake.uuid4(), "name": fake.word().capitalize() + f" ({random.randint(50, 500)}mg)", "frequency": random.choice(["Once daily", "Twice daily", "Every 8 hours"])} for _ in range(NUM_MEDICATIONS)]

appointments = [{"appointment_id": fake.uuid4(), "patient_id": random.choice(patients)["patient_id"], "doctor_id": random.choice(doctors)["doctor_id"], "date": fake.date_between(start_date="today", end_date="+30d"), "time": f"{random.randint(9, 11)}:{random.choice(['00', '30'])} {random.choice(['AM', 'PM'])}"} for _ in range(NUM_APPOINTMENTS)]


In [28]:
doctors_df.head()

,doctor_id,name,specialization,availability
0,9356b689-e0ab-4aae-88b8-72be6ad70df5,Beth Moran,Neurologist,Tue 12 AM to 1 PM
1,d3f358cd-d163-4981-8a6c-a0ca49358a57,Amy Rivera,Cardiologist,Mon 11 AM to 1 PM
2,7b27f1a7-3f2c-4635-bf1f-bd2073b5d9fc,Marissa Klein MD,Neurologist,Thu 9 AM to 2 PM
3,607cf353-89ef-4df3-9241-fec66be0d6f6,Carol Barnes,Dermatologist,Mon 9 AM to 2 PM
4,94db5c41-fbd4-4f83-9801-ecf8b4002ccb,Valerie Thompson,Cardiologist,Fri 10 AM to 5 PM


In [29]:
patients_df = pd.DataFrame(patients)
doctors_df = pd.DataFrame(doctors)
medications_df = pd.DataFrame(medications)
appointments_df = pd.DataFrame(appointments)
symptoms_df = pd.DataFrame(symptoms_conditions)


In [44]:
patients_df = pd.read_csv("https://github.com/Palakjhamnani/Health-care-chatbot/raw/main/patients.csv")
doctors_df = pd.read_csv("https://github.com/Palakjhamnani/Health-care-chatbot/raw/main/doctors.csv")
medications_df = pd.read_csv("https://github.com/Palakjhamnani/Health-care-chatbot/raw/main/medications.csv")
appointments_df = pd.read_csv("https://github.com/Palakjhamnani/Health-care-chatbot/raw/main/appointments.csv")
symptoms_df = pd.read_csv("https://github.com/Palakjhamnani/Health-care-chatbot/raw/main/symptoms_conditions.csv")


In [45]:
patient_id_mapping = {old_id: new_id for new_id, old_id in enumerate(patients_df["patient_id"].unique(), start=1)}
patients_df["patient_id"] = patients_df["patient_id"].map(patient_id_mapping)
appointments_df["patient_id"] = appointments_df["patient_id"].map(patient_id_mapping)

# Save updated DataFrames back to CSV
patients_df.to_csv("patients_updated.csv", index=False)
appointments_df.to_csv("appointments_updated.csv", index=False)

print("Patient IDs have been updated to sequential numbers.")


Patient IDs have been updated to sequential numbers.


In [46]:
doctor_id_mapping = {old_id: new_id for new_id, old_id in enumerate(doctors_df["doctor_id"].unique(), start=1)}
doctors_df["doctor_id"] = doctors_df["doctor_id"].map(doctor_id_mapping)
appointments_df["doctor_id"] = appointments_df["doctor_id"].map(doctor_id_mapping)

# Save the updated DataFrames back to CSV

In [60]:
def get_patient_record(patient_name):
    patient_name = patient_name.lower()
    matches = patients_df[patients_df["name"].str.lower().str.contains(patient_name)]

    if not matches.empty:
        return f"\nPatient Record:\n{matches.to_string(index=False)}"
    return "No patient record found."


def analyze_symptoms(symptoms):
    mock_conditions = {
        "fever and cough": "You might have the flu. Please consult a doctor.",
        "chest pain": "This could be a sign of a heart problem. Seek immediate attention."
    }
    for key, value in mock_conditions.items():
        if key in symptoms.lower():
            return value
    return "I'm not sure about your symptoms. Please consult a doctor."


def book_appointment(patient_name, doctor_name, date, time):
    global appointments_df  # Declare appointments_df as global

    patient_match = patients_df[patients_df["name"].str.lower().str.contains(patient_name.lower())]
    doctor_match = doctors_df[doctors_df["name"].str.lower().str.contains(doctor_name.lower())]

    if patient_match.empty or doctor_match.empty:
        return "Invalid patient or doctor name. Please try again."

    patient = patient_match.iloc[0]
    doctor = doctor_match.iloc[0]

    # Create a new DataFrame for the new appointment
    new_appointment = pd.DataFrame([{
        "appointment_id": len(appointments_df) + 1,
        "patient_id": patient["patient_id"],
        "doctor_id": doctor["doctor_id"],
        "date": date,
        "time": time,
    }])

    # Use pd.concat to add the new appointment to the appointments_df
    appointments_df = pd.concat([appointments_df, new_appointment], ignore_index=True)

    return f"Appointment booked with Dr. {doctor['name']} on {date} at {time}."

def medication_reminder(patient_name):
    patient_match = patients_df[patients_df["name"].str.lower().str.contains(patient_name.lower())]
    if not patient_match.empty:
        return f"Reminder: {patient_match.iloc[0]['name']}, take your prescribed medication at the scheduled time."
    return "No medication data found for the specified patient."


In [56]:
patients_df.head()

,patient_id,name,age,gender,medical_history,allergies
0,1,Kyle Williams,31,Male,"['Allergy', 'Flu']",Peanuts
1,2,Jeremy Salinas,29,Male,"['Migraine', 'Tonsillitis', 'Heart Attack']",NaN
2,3,Eric Moore,90,Male,"['Allergy', 'Anemia']",Penicillin
3,4,Victoria Rasmussen,58,Male,"['Anemia', 'Allergy']",Pollen
4,5,Jason Mcintyre,23,Male,['Flu'],Peanuts


In [57]:
doctors_df.head()

,doctor_id,name,specialization,availability
0,1,Frances Hernandez,General Physician,Tue 11 AM to 2 PM
1,2,Tina Brooks,Dermatologist,Wed 9 AM to 4 PM
2,3,Gregory Wilson,General Physician,Tue 10 AM to 2 PM
3,4,Janet Hunter,Neurologist,Thu 10 AM to 4 PM
4,5,Rachel White,Cardiologist,Fri 12 AM to 4 PM


In [62]:
def chatbot():
    print("\nWelcome to the Healthcare Assistant Chatbot!")
    patient_name = input("Please enter your name: ").strip()  # Ask for patient's name once

    while True:
        print("\nHow can I assist you today? Select an option:")
        print("1. View Patient Record")
        print("2. Analyze Symptoms")
        print("3. Book an Appointment")
        print("4. Medication Reminder")
        print("5. Find Doctor by Specialization")
        print("6. Exit")

        choice = input("Enter your choice (1-6): ").strip()

        if choice == "1":
            print(get_patient_record(patient_name))

        elif choice == "2":
            symptoms = input("Describe your symptoms: ").strip()
            print(analyze_symptoms(symptoms))

        elif choice == "3":
            # Suggest 5 doctors to choose from for booking an appointment
            print("\nHere are 5 doctors you can choose from:")
            available_doctors = doctors_df.sample(5)  # Randomly choose 5 doctors
            for idx, doctor in available_doctors.iterrows():
                print(f"{doctor['name']} - {doctor['specialization']} - Available on: {doctor['availability']}")

            doctor_name = input("\nChoose a doctor from the list above by entering their name: ").strip()
            date = input("Enter the appointment date (YYYY-MM-DD): ").strip()
            time = input("Enter the appointment time (e.g., 10:00 AM): ").strip()
            print(book_appointment(patient_name, doctor_name, date, time))

        elif choice == "4":
            print(medication_reminder(patient_name))

        elif choice == "5":
            specialization = input("Enter the doctor's specialization (e.g., Cardiology): ").strip().lower()
            matched_doctors = doctors_df[doctors_df["specialization"].str.lower() == specialization]
            if not matched_doctors.empty:
                print("\nDoctors matching your search:")
                print(matched_doctors[["name", "specialization"]].to_string(index=False))
            else:
                print("No doctors found for the specified specialization.")

        elif choice == "6":
            print("Goodbye! Stay healthy.")
            break

        else:
            print("Invalid choice. Please try again.")


# Start the chatbot
chatbot()


Welcome to the Healthcare Assistant Chatbot!
Please enter your name: Kyle Williams	

How can I assist you today? Select an option:
1. View Patient Record
2. Analyze Symptoms
3. Book an Appointment
4. Medication Reminder
5. Find Doctor by Specialization
6. Exit
Enter your choice (1-6): 3

Here are 5 doctors you can choose from:
Michael Ho - General Physician - Available on: Mon 12 AM to 5 PM
Amanda Petty DDS - Dermatologist - Available on: Wed 10 AM to 2 PM
Rachel White - Cardiologist - Available on: Fri 12 AM to 4 PM
Frances Hernandez - General Physician - Available on: Tue 11 AM to 2 PM
Donna Butler - Neurologist - Available on: Thu 10 AM to 1 PM

Choose a doctor from the list above by entering their name: Donna Butler
Enter the appointment date (YYYY-MM-DD): 2024-12-15
Enter the appointment time (e.g., 10:00 AM): 10:00 AM
Appointment booked with Dr. Donna Butler on 2024-12-15 at 10:00 AM.

How can I assist you today? Select an option:
1. View Patient Record
2. Analyze Symptoms
3. Bo